## Executando Modelos Preditivos

## Imports Necessários

### Imports Pacotes Externos

In [2]:
import re
import math
import json
import pickle
import numbers
import os
import os.path
import sys
import glob
import itertools
import subprocess
import warnings

import importlib
import joblib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from datetime import datetime
from scipy.interpolate import interp1d
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import mean_squared_error, r2_score
from IPython.display import display, clear_output

import pathlib

### Imports Módulos Internos

In [1]:
from pelopt.utils.utils import *
from pelopt.utils.predictive_module import *
from pelopt.utils.model_module import *
from pelopt.usina.df_detailed import *
from pelopt.usina.predictive_module import *
from pelopt.utils.operations import operations

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/17 06:48:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
n_usina = 8
us_sufix = f'{n_usina:02d}'
solver_path = f'us{n_usina:1d}'

# data_file = f'.{os.path.sep}tmp{os.path.sep}{solver_path}{os.path.sep}datasets.joblib'
data_file = 'datasets.joblib'
datasets = joblib.load(data_file)

In [4]:
print(datasets['abrasao'].index.min())
print(datasets['abrasao'].index.max())
datasets['abrasao'].describe()

2021-07-15 08:00:00
2022-11-30 21:00:00


variavel,soma balanca minerio misturador,media disco de pelotamento,media temp pelotas,soma balanca bentonita misturador,media tm,soma balanca retorno correia,QUIM_CAO_PP_L@08PR,antracito,PESO1_I@08MO-BW-813I-03M1,vazao_antracito,...,POSI1_D@08QU-VD-853I-04M1,POSI1_D@08QU-VD-853I-20M1,DESV MEDIO ALT CAMADA,TEMP1_C@08QU-PF-852I-05M1,TEMP1_I@08QU-QU-855I-GQ06,TEMP1_I@08QU-WB-851I-08,PRES1_I@08QU-WB-851I-12,PRES1_I@08QU-WB-851I-09,status,"ABRA_-0,5_PQ_L@08QU"
count,9482.000000,9482.000000,9482.000000,9482.000000,9482.000000,9482.000000,7622.000000,8565.000000,9001.000000,3762.000000,...,9482.000000,9465.0,8524.000000,8704.000000,8387.000000,8563.000000,8532.000000,8532.000000,9482.000000,8491.000000
mean,871.094021,5.129901,87.795406,4.184574,10.795632,169.452395,2.603866,16.230987,5.287340,29.450091,...,0.716832,0.0,10.663374,148.201929,1069.365062,98.394989,-369.945373,-348.997325,0.955916,5.284122
std,258.821612,1.263355,22.715444,1.311646,2.742501,56.446077,0.202016,1.295814,10.128619,6.189375,...,0.450561,0.0,3.722056,28.826884,73.901717,9.296152,55.674571,53.399108,0.205292,0.702976
min,0.000000,-0.001047,20.365000,-11.344281,0.000000,-5.690000,2.130000,13.400000,0.000000,18.790000,...,0.000000,0.0,4.624324,106.040000,910.659851,86.010000,-488.500000,-461.120000,0.000000,4.160000
25%,757.692500,4.760000,71.336250,3.620000,10.068271,133.780000,2.460000,15.300000,0.000000,23.684689,...,0.000000,0.0,7.711571,122.507500,1027.400000,91.020000,-410.330000,-387.385000,1.000000,4.730000
50%,899.025000,5.301250,85.115000,4.290000,11.210000,174.960000,2.610000,16.300000,0.000000,30.255000,...,1.000000,0.0,10.107828,143.440000,1059.600000,95.550000,-365.960000,-345.625000,1.000000,5.150000
75%,1060.775000,5.870833,100.325000,5.120000,12.388979,213.500000,2.760000,17.200000,0.000000,34.847500,...,1.000000,0.0,13.163105,170.865000,1124.400000,104.205000,-325.590000,-308.077500,1.000000,5.790000
max,1408.890000,6.863333,260.660000,7.410000,14.573333,739.470000,2.980000,18.900000,35.132830,39.830000,...,1.000000,0.0,20.496106,197.180000,1204.200000,116.210000,-255.848083,-242.130000,1.000000,6.510000


In [5]:
model = 'abrasao'
df = datasets[model]
datasets[model] = df[(df[df.columns[-1]] >= 4.5) & (df[df.columns[-1]] < 6)]

model = 'compressao'
df = datasets[model]
datasets[model] = df[(df[df.columns[-1]] >= 230) & (df[df.columns[-1]] < 330)]


models = {
    'ridge': {
        'model': Ridge,
        'params': {
            # [100.0, 10.0, 1.0, 0.1, 0.01, 0.001]
            'alpha': [100/10**i for i in range(6)],
        }
    },
}

cv_n = 3
cv_size = 0.3

In [6]:
bad_models = ['finos', 'SE PP', 'particulados1', 'particulados2',
              'particulados3', 'energia_prensa', 'custo_SE PP',
              'produtividade filtragem', 'taxarp']

low_error_models = [c for c in datasets.keys() if c not in bad_models]
main_models = ['abrasao', 'basicidade', 'distribuicao gran', 'GRAN PR',
               'SE PR', 'energia_forno', 'umidade']
model_group = datasets.keys()

In [10]:
qualidade = 'abrasao'
print(f"{f' {qualidade.capitalize()} ':=^50}")


==================== Abrasao =====================


In [11]:
def apply_naive_model(X, y):
    X_test = X[X.index > test_set_date]
    X_train = X[X.index < test_set_date]
    y_test = y[y.index > test_set_date]
    y_train = y[y.index < test_set_date]    
    
    predicted = [y_train.mean()]*len(y_test)
    
    print(len(y_test), len(predicted))
    mse = mean_squared_error(y_test, predicted)
    mape = np.mean(np.abs(y_test - predicted) / np.abs(y_test)) * 100
    r2 = r2_score(y_test, predicted)
    mse, mape, r2 = round(mse, 3), round(mape, 3), round(r2, 3)
    
    return mse, mape, r2


def get_prediction(datasets, model_group, set_type, apply_baseline=False):
    """
    set_type: validation or test
    """
    results, scalers, limits = {}, {}, {}
    nive_concat = pd.DataFrame()

    scalers, all_df = operations.define_real_scalers(datasets)

    df_target_dict = {}
    for qualidade in model_group:

        # for qualidade in ['abrasao']:
        if qualidade.startswith("cm"):
            continue
        print(f"{f' {qualidade.capitalize()} ':=^50}")

        results[qualidade] = []
        df_train = datasets[qualidade].copy()
        col_target = df_train.columns[-1]
        print("DEBUG - Target: ", col_target)
        for c in ["GANHO PRENSA - US8"]:
            if c in df_train.columns[:-1]:
                df_train[c] = np.log(df_train[c])
        # Aplicando filtros sobre o df_train
        print(df_train.shape)
        df_train = mod_filtros(df_train, qualidade, col_target)
        print(df_train.shape)

        # Passando por todas as colunas do df (exceto a target)
        for t in df_train.columns[:-1]:
            # Definindo as colunas que serão ignoradas
            if (
                t in ["status", "ProducaoPQ_Moagem"]
                or t.startswith("qtde")
                or t.startswith("SOMA FUNC")
            ):
                df_train[t].fillna(df_train[t].mode()[0], inplace=True)
                continue

            # Aplicando tratamento nos dados (fillna e rolling)
            df_train = tratando_dados(
                df_train,
                qualidade,
                t,
                restrict_to=[
                    "abrasao",
                    "basicidade",
                    "distribuicao gran",
                    "finos",
                    "GRAN PR",
                    "SE PR",
                    "dens_moinho_1",
                    "dens_moinho_2",
                    "dens_moinho_3",
                ],
            )

            # Gerando o scaler dos NIVE
            if (
                t
                in [
                    "NIVE1_C@08QU-FR-851I-01M1",
                    "NIVE2_I@08QU-FR-851I-01M1",
                    "NIVE3_I@08QU-FR-851I-01M1",
                    "NIVE4_I@08QU-FR-851I-01M1",
                    "NIVE5_I@08QU-FR-851I-01M1",
                    "NIVE6_I@08QU-FR-851I-01M1",
                ]
                and t not in scalers
            ):
                nive_concat = pd.concat([nive_concat, df_train[t]], ignore_index=True)

            # Criando os scalers dos atributos
            if t not in scalers:
                print(
                    (
                        "tag entrou no if. Dai faz um fit nela pq deve ter sido"
                        " ignorado na etapa de geracao dos scalers. Parece uma"
                        " gambiarra do Leandro"
                    ),
                    t,
                )
                scalers[t] = MinMaxScaler()

                tmp = df_train[[t]].copy()
                if t.startswith("DENS1_C@08HO-BP"):
                    tmp[t][0] = 0
                scalers[t].fit(tmp.astype(float))

            # Escalonando os valores

            # ATENCAO: DATA LEAKAGE OCORRE NESSE MOMENTO
            # Os valores de filtragem sao todos negativos, aqui ele ta cortando
            # eles ai o dataframe fica zerado
            df_train[t] = scalers[t].transform(df_train[[t]])

        # Selecionando apenas os elementos válidos
        print(f"Pre-remoção status zerado: {df_train.shape[0]:,} x {df_train.shape[1]:,}")
        if "status" in df_train.columns:
            df_train = df_train.query("status == 1").drop("status", axis=1)
        print(f"Pós-remoção status zerado: {df_train.shape[0]:,} x {df_train.shape[1]:,}")

        # Aplicando um filtro de elementos
        if "ProducaoPQ_Moagem" in df_train:
            p1 = df_train["ProducaoPQ_Moagem"] >= 700
            p2 = df_train["ProducaoPQ_Moagem"] <= 1000
            df_train = df_train[p1 & p2].drop("ProducaoPQ_Moagem", axis=1)

        # Aplicando filtro de target maior que 0
        if qualidade.startswith("energia") or qualidade in ["gas"]:
            df_train = df_train[df_train[col_target] > 0]
            print(df_train.shape)

        df_train = df_train.replace([np.inf, -np.inf], np.nan)
        df_train = (
            df_train.fillna(method="bfill")
            .fillna(method="ffill")
            .fillna(method="bfill")
            .fillna(0)
        )

        # Gerando o treino e target
        df_train = df_train.dropna(subset=[col_target])
        y_train = df_train[col_target]
        df_train = df_train.drop(col_target, axis=1)

        # Dados para os limites de cada atributo
        for c in df_train.columns:
            limits[c] = df_train[c]

        # Gerando o threshold do cross fold validation
        print(df_train.shape)
        ix = df_train.index.sort_values()
        if set_type == "validation":
            cv_thresholds = [
                ix[int(len(ix) * (1 - i * cv_size / cv_n)) - 1] for i in range(cv_n + 1)
            ][::-1]
        elif set_type == "test":
            cv_thresholds = [test_set_date, ix[-1]]

        # Executando os modelos
        for method, model_conf in models.items():
            print(method)
            if apply_baseline == False:
                results[qualidade] = apply_model(
                    model_conf,
                    qualidade,
                    method,
                    df_train,
                    y_train,
                    solver_path,
                    validation=manual_kfold_validation,
                    plot_charts=plot_charts,
                    selfTrain=False,
                    param_validation={
                        "cv_thresholds": cv_thresholds,
                        "qualidade": qualidade,
                    },
                )
            elif apply_baseline == True:
                results[qualidade] = apply_naive_model(df_train, y_train)
    df_target_dict[qualidade] = y_train
    return results, scalers, limits, nive_concat, df_target_dict


## Modelos Baseline

In [12]:
test_set_date = '2022-06-01'
baseline_results, _, _, _, _ = get_prediction(datasets, model_group, set_type='test', apply_baseline = True)

==================== Abrasao =====================
DEBUG - Target:  ABRA_-0,5_PQ_L@08QU
(5669, 163)
(5669, 163)
tag entrou no if. Dai faz um fit nela pq deve ter sido ignorado na etapa de geracao dos scalers. Parece uma gambiarra do Leandro NIVE1_C@08QU-FR-851I-01M1
Pre-remoção status zerado: 5,669 x 163
Pós-remoção status zerado: 5,503 x 162
(5503, 161)
ridge
1461 1461
=================== Compressao ===================
DEBUG - Target:  COMP_MCOMP_PQ_L@08QU
(6708, 163)
(6708, 163)
Pre-remoção status zerado: 6,708 x 163
Pós-remoção status zerado: 6,369 x 162
(6369, 161)
ridge
2498 2498
=================== Basicidade ===================
DEBUG - Target:  QUIM_BAS2_PQ_L@08QU
(9482, 9)
(8251, 9)
Pre-remoção status zerado: 8,251 x 9
Pós-remoção status zerado: 7,918 x 8
(7918, 7)
ridge
2595 2595
===================== Se pp ======================
DEBUG - Target:  GANHO PRENSA - US8
(9482, 81)
(5014, 81)
tag entrou no if. Dai faz um fit nela pq deve ter sido ignorado na etapa de geracao dos sca

## Modelos Finais

In [38]:
models_results, scalers, limits, nive_concat, df_target_dict = get_prediction(
    datasets, model_group, set_type='validation', apply_baseline = False
)

==================== Abrasao =====================
DEBUG - Target:  ABRA_-0,5_PQ_L@08QU
(5669, 163)
(5669, 163)
tag entrou no if. Dai faz um fit nela pq deve ter sido ignorado na etapa de geracao dos scalers. Parece uma gambiarra do Leandro NIVE1_C@08QU-FR-851I-01M1


[2023-01-17 07:05:36,987 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:301] - ###  + ridge


Pre-remoção status zerado: 5,669 x 163
Pós-remoção status zerado: 5,503 x 162
(5503, 161)
ridge


[2023-01-17 07:05:37,765 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.177 - MAPE:  6.667% - r2: -0.109 - r:  0.120 - r2_train:  0.272 - r2_train_adj:  0.244
[2023-01-17 07:05:38,554 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.223 - MAPE:  7.359% - r2: -0.394 - r:  0.127 - r2_train:  0.345 - r2_train_adj:  0.320
[2023-01-17 07:05:39,339 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.252 - MAPE:  7.860% - r2: -0.578 - r:  0.150 - r2_train:  0.383 - r2_train_adj:  0.359
[2023-01-17 07:05:40,050 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.244 - MAPE:

=================== Compressao ===================
DEBUG - Target:  COMP_MCOMP_PQ_L@08QU
(6708, 163)
(6708, 163)


...

[2023-01-17 07:07:40,162 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.025 - MAPE:    inf% - r2: -0.044 - r:  0.142 - r2_train:  0.203 - r2_train_adj:  0.185
[2023-01-17 07:07:40,866 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.028 - MAPE:    inf% - r2: -0.151 - r:  0.116 - r2_train:  0.261 - r2_train_adj:  0.244
[2023-01-17 07:07:41,559 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.034 - MAPE:    inf% - r2: -0.388 - r:  0.095 - r2_train:  0.286 - r2_train_adj:  0.270
[2023-01-17 07:07:42,273 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  0.040 - MAPE:

================== Relacao gran ==================
DEBUG - Target:  rel_gran
(9482, 357)
(9482, 357)


[2023-01-17 07:07:54,661 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:301] - ###  + ridge


Pre-remoção status zerado: 9,482 x 357
Pós-remoção status zerado: 9,064 x 356
(9064, 355)
ridge


[2023-01-17 07:07:55,525 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  1.654 - MAPE: 39.639% - r2: -0.465 - r:  0.060 - r2_train:  0.337 - r2_train_adj:  0.302
[2023-01-17 07:07:56,295 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  1.834 - MAPE: 40.637% - r2: -0.748 - r:  0.003 - r2_train:  0.403 - r2_train_adj:  0.372
[2023-01-17 07:07:57,066 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  2.098 - MAPE: 44.705% - r2: -1.097 - r: -0.013 - r2_train:  0.421 - r2_train_adj:  0.391
[2023-01-17 07:07:57,837 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/model_module.py:model_module:376] - alph=Y.............. - MSE:  2.174 - MAPE:

In [15]:
for model, df in datasets.items():
    for col in df.columns:
        if col.startswith('GRAN'):
            print(model, col, sep=' - ')


abrasao - GRAN_OCS_TM@08PE-BD-840I-03
abrasao - GRAN_OCS_TM@08PE-BD-840I-06
abrasao - GRAN_OCS_TM@08PE-BD-840I-01
abrasao - GRAN_OCS_TM@08PE-BD-840I-02
abrasao - GRAN_OCS_TM@08PE-BD-840I-11
abrasao - GRAN_OCS_TM@08PE-BD-840I-05
abrasao - GRAN_OCS_TM@08PE-BD-840I-12
abrasao - GRAN_OCS_TM@08PE-BD-840I-08
abrasao - GRAN_OCS_TM@08PE-BD-840I-10
abrasao - GRAN_OCS_TM@08PE-BD-840I-07
abrasao - GRAN_OCS_TM@08PE-BD-840I-09
abrasao - GRAN_OCS_TM@08PE-BD-840I-04
compressao - GRAN_OCS_TM@08PE-BD-840I-03
compressao - GRAN_OCS_TM@08PE-BD-840I-06
compressao - GRAN_OCS_TM@08PE-BD-840I-01
compressao - GRAN_OCS_TM@08PE-BD-840I-02
compressao - GRAN_OCS_TM@08PE-BD-840I-11
compressao - GRAN_OCS_TM@08PE-BD-840I-05
compressao - GRAN_OCS_TM@08PE-BD-840I-12
compressao - GRAN_OCS_TM@08PE-BD-840I-08
compressao - GRAN_OCS_TM@08PE-BD-840I-10
compressao - GRAN_OCS_TM@08PE-BD-840I-07
compressao - GRAN_OCS_TM@08PE-BD-840I-09
compressao - GRAN_OCS_TM@08PE-BD-840I-04
SE PP - GRAN_-0,045_PR_L@08FI
GRAN PR - GRAN_-0,045_

In [39]:
models_features = limits

In [41]:
temp_dict = {}
models_coeficients = {}

for model in models_results.keys():

    best = retrieve_best_model(models_results, model, 'mape')

    temp_dict[model] = {
        'mape': round(best['metrics']['mape'], 3),
        'r2': round(best['metrics']['r2'], 3)
    }
    show_best_result(model, best)

    coefficients = {
        col : coef for col, coef in zip(best['columns'], best['model'].coef_)
    }
    models_coeficients[model] = coefficients

[2023-01-17 07:09:03,281 | INFO | MainProcess | /Users/erikingwersen/Documents/GitHub/peloptimize/src/pelopt/utils/predictive_module.py:predictive_module:28] - abrasao                   ridge        6.67%  -0.11    0.12


TypeError: 'TypeError' object is not iterable